In [11]:
# Import Libraries
import pyautogui as p # used to move the mouse and click and enter keystrokes
import time # used for wait function so clicks are not too fast
import random as r # used for randomizing click times and locations
from datetime import datetime as date # used to find the right spot on the calendar
from datetime import timedelta # used to run the code a week before the desired slot time right when bookings open
from twilio.rest import Client # this is a free service to be able to automatically text my phone when the code either books or fails
import pause # used to delay running the code till the booking slot actually opens
from selenium import webdriver

# Set Me!
booking_url = 'https://app.rockgympro.com/b/widget/?a=offering&offering_guid=6fa9139cc3584fc0a5662a5c36d68958&widget_guid=78c7ab47bbe04fa59fe0a4889f4cda4c&random=6063894dc432e&iframeid=&mode=p'
booking_month = 'May'
day_to_book = 'Monday' # Notice exact formatting in day dictionary!
date_to_book = '24'
time_to_book = '2 PM to  4 PM' #be careful with format (single space between #_PM, but double space between to__#)
month = 5 # only need this for the pause function 
year = 2021 # only need this for the pause function
mode = 1 # Set to: 0 if you are waiting for somebody to cancel, 1 if you are trying to book right once the slot opens up

# Dictionaries, Calculations, and Things Not To Touch
today = date.today()
this_month = today.strftime("%B")
# Determines Which Row Of The Calendar (treated like an array for indexing) Our Booking Day Is
if booking_month == this_month:
    ref_day = '01 ' + date.today().strftime('%m %Y')
    day = date.strptime(ref_day, '%d %m %Y').weekday()
    row = int((int(date_to_book) + int(day) - 1)/7)
elif booking_month != this_month:
    next_month = date.today() + timedelta(weeks =+ 1)
    ref_day = '01 ' + next_month.strftime('%m %Y')
    day = date.strptime(ref_day, '%d %m %Y').weekday()
    row = int((int(date_to_book) + int(day) - 1)/7)
    
client = Client("ACa3f2f8227c2375850b8a8dd741b7d665", "81e2dedcf51b88ec9031e6e014e5e039")
time_slot_dict_MTF = {'7 AM to  9 AM':1, '8 AM to  10 AM':2, '9 AM to  11 AM':3, '10 AM to  12 PM':4, '11 AM to  1 PM':5,
                     '12 PM to  2 PM':6, '1 PM to  3 PM':7, '2 PM to  4 PM':8, '3 PM to  5 PM':9, '4 PM to  6 PM':10,
                     '5 PM to  7 PM':11, '6 PM to  8 PM':12, '7 PM to  9 PM':13, '8 PM to  10 PM':14, '9 PM to  11 PM':15}
time_slot_dict_WTh = {'12 PM to  2 PM':1, '1 PM to  3 PM':2, '2 PM to  4 PM':3, '3 PM to  5 PM':4, '4 PM to  6 PM':5,
                     '5 PM to  7 PM':6, '6 PM to  8 PM':7, '7 PM to  9 PM':8, '8 PM to  10 PM':9, '9 PM to  11 PM':10}
time_slot_dict_SS = {'10 AM to  12 PM':1, '11 AM to  1 PM':2, '12 PM to  2 PM':3, '1 PM to  3 PM':4, '2 PM to  4 PM':5, 
                     '3 PM to  5 PM':6, '4 PM to  6 PM':7, '5 PM to  7 PM':8, '6 PM to  8 PM':9, '7 PM to  9 PM':10, 
                     '8 PM to  10 PM':11}
day_dict = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
calendar_dict_x = {'Monday':[493,520], 'Tuesday':[532,554], 'Wednesday':[566,593], 'Thursday':[605,629], 'Friday':[642,665], 'Saturday':[675,702], 'Sunday':[711,738]}
calendar_dict_y = [[800,817],[825,842],[851,867],[875,891],[900,915],[921,938]]
MTF_y = {'7 AM to  9 AM':[400,418], '8 AM to  10 AM':[437,458], '9 AM to  11 AM':[477,496], '10 AM to  12 PM':[516,535], '11 AM to  1 PM':[554,576],
                     '12 PM to  2 PM':[596,613], '1 PM to  3 PM':[635,651], '2 PM to  4 PM':[671,688], '3 PM to  5 PM':[713,728], '4 PM to  6 PM':[749,769],
                     '5 PM to  7 PM':[789,807], '6 PM to  8 PM':[829,846], '7 PM to  9 PM':[869,885], '8 PM to  10 PM':[907,928], '9 PM to  11 PM':[945,966]}
WTh_y = {'12 PM to  2 PM':[596,613], '1 PM to  3 PM':[635,651], '2 PM to  4 PM':[671,688], '3 PM to  5 PM':[713,728], '4 PM to  6 PM':[749,769],
                     '5 PM to  7 PM':[789,807], '6 PM to  8 PM':[829,846], '7 PM to  9 PM':[869,885], '8 PM to  10 PM':[907,928], '9 PM to  11 PM':[945,966]}
SS_y = {'10 AM to  12 PM':[554,576], '11 AM to  1 PM':[596,613], '12 PM to  2 PM':[635,651], '1 PM to  3 PM':[671,688], '2 PM to  4 PM':[713,728], 
                     '3 PM to  5 PM':[749,769], '4 PM to  6 PM':[789,807], '5 PM to  7 PM':[829,846], '6 PM to  8 PM':[869,885], '7 PM to  9 PM':[907,928], 
                     '8 PM to  10 PM':[945,966]}
#slots_x = [775,831] 
slots_x = [899,951] # the x location when the booking is in the current week (I dont think this is relevant anymore with the way i implemented the waiting for cancellations)
default_size = [1920,1080] # this is the default monitor size i used to get all the measurements above and used as a reference to scale mouse movements

# Finds The Time The Code Should Run (One Week Before The Desired Booking Slot Since That's When Booking Opens)
if time_to_book[2] == 'P':
    time_to_go = int(time_to_book[0]) + 12
elif time_to_book[2] == 'A':
    time_to_go = int(time_to_book[0])
elif time_to_book[3] == 'P': # the case where it is 12 PM ie Noon
    time_to_go = int(time_to_book[0:2])
elif time_to_book[3] == 'A':
    time_to_go = int(time_to_book[0:2])
else: 
    print('error in the date')
    client.messages.create(to = "+17789852567", from_ = "+12564004669", body = "Date Error")

year_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%Y')
month_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%m')
date_before = (date(year, month, int(date_to_book)) - timedelta(days=7)).strftime('%d')

# print(x_scale,y_scale)
print(row)
# print(year_before, month_before, date_before, time_to_go)

def move_n_click(xy):
    nextx = r.randint(xy[0],xy[1])
    nexty = r.randint(xy[2],xy[3])
    p.moveTo(nextx, nexty, duration = 0.3-r.randint(0,2)/10)
    p.click(nextx,nexty)

4


In [13]:
## This Cell Executes All the Mouse Movements To Actually Book
# opens a second tab in this chrome window
new_tab = [265,280,10,24]
move_n_click(new_tab)
time.sleep(0.5)

# enters the url to the hive booking website
url = 'https://app.rockgympro.com/b/widget/?a=list&&widget_guid=78c7ab47bbe04fa59fe0a4889f4cda4c&random=6075d2d653092&iframeid=&mode=p'
p.typewrite(url)
p.typewrite(["enter"])
time.sleep(1.5)

if mode == 1:
    # waits until booking opens for the slot we want
    pause.until(date(int(year_before), int(month_before), int(date_before), time_to_go))
    time.sleep(0.3)
    print('yeppers')
elif mode == 0:
    # Opens An Automated Chrome Browser
    PATH = "C:\chromedriver.exe" # path to chromedriver
    driver = webdriver.Chrome(PATH) # open automated browser window
    driver.maximize_window() # resize window

    # Opens The Booking Website
    driver.get(booking_url)
    time.sleep(1) # wait for website to fully load

    while True:
        try:
            # Determine if booking is in the right month and if not change it
            if booking_month == this_month:
                pass
            else:
                # Click the arrow on the calendar to go to the next month
                btn_nextmonth = driver.find_element_by_xpath('//*[@id="start_date_calendar"]/div/div/a[2]')
                btn_nextmonth.click()

            # Chooses the date on the calendar based on row calculation and weekday column
            btn_date = driver.find_element_by_xpath(f'//*[@id="start_date_calendar"]/div/table/tbody/tr[{row + 1}]/td[{day_dict[day_to_book]}]/a')
            btn_date.click()

            if day_to_book == 'Monday' or day_to_book == 'Tuesday' or day_to_book == 'Friday':
                xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_MTF[time_to_book]}]/td[4]/a'
            elif day_to_book == 'Wednesday' or day_to_book == 'Thursday':
                xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_WTh[time_to_book]}]/td[4]/a'
            elif day_to_book == 'Saturday' or day_to_book == 'Sunday':
                xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_SS[time_to_book]}]/td[4]/a'
            else: print('Error: No Matching Day')

            p.moveTo(500, 500, duration = 0.1)
            p.scroll(-1000)
            element = driver.find_element_by_xpath(xpath_slot)
            page = driver.find_element_by_xpath('/html')
            h = page.size['height']
            # get absolute x value
            x1 = element.location['x'] + 3# add a buffer of 3 pixels
            x2 = x1 + element.size['width'] - 6

            # get y value (this is relative y value - meaning y value with in browser)
            y = element.location['y'] 
            y1 = 1080 - (h - y + 30) + 3 # 30 is the height of the taskbar on my computer, 1080 is the full height of my monitor
            y2 = y1 + element.size['height'] - 6
            # print the absolute x and y values
            #print (x1, x2, y1, y2)
            #print('passed')
            driver.quit()
            break
        except:
            driver.refresh()
            
# clicks on the info and bookings button
info_button = [691,879, 548,575]
move_n_click(info_button)
time.sleep(0.8)

# scrolls down a bit to where the members and calendar selection are
p.scroll(-230)
time.sleep(0.3)

# adds one member (can add clicks as the number of members increases but currently only can book one)
#p.locateCenterOnScreen('plusbutton.png')
members = [879,900,730,751]
move_n_click(members)
time.sleep(0.2)

if booking_month == this_month:
    pass
else:
    #click next month arrow
    next_arrow = [716,736,734,755]
    move_n_click(next_arrow)
    time.sleep(0.3)

# selects the day in the calendar based on the row and the column
dayx = r.randint(calendar_dict_x[day_to_book][0],calendar_dict_x[day_to_book][1])
dayy = r.randint(calendar_dict_y[row][0],calendar_dict_y[row][1])
p.moveTo(dayx, dayy, duration = 0.4-r.randint(0,3)/10)
p.click(dayx,dayy)
time.sleep(0.2)

# scroll all the way to the bottom of the page
p.scroll(-500)
time.sleep(0.3)

if mode == 1:
    # selects the desired booking slot (number of available rows depends on the day since some days have less hours)
    if day_to_book == 'Monday' or day_to_book == 'Tuesday' or day_to_book == 'Friday':
        slotx = r.randint(slots_x[0],slots_x[1])
        sloty = r.randint(MTF_y[time_to_book][0],MTF_y[time_to_book][1])
        p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
        p.click(slotx,sloty)
        time.sleep(0.5)
    elif day_to_book == 'Wednesday' or day_to_book == 'Thursday':
        slotx = r.randint(slots_x[0],slots_x[1])
        sloty = r.randint(WTh_y[time_to_book][0],WTh_y[time_to_book][1])
        p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
        p.click(slotx,sloty)
        time.sleep(0.5)
    elif day_to_book == 'Saturday' or day_to_book == 'Sunday':
        slotx = r.randint(slots_x[0],slots_x[1])
        sloty = r.randint(SS_y[time_to_book][0],SS_y[time_to_book][1])
        p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
        p.click(slotx,sloty)
        time.sleep(0.5)
    else: print('error')
elif mode == 0:
    # selects the desired booking slot (number of available rows depends on the day since some days have less hours)
    slotx = r.randint(x1,x2)
    sloty = r.randint(y1,y2)
    p.moveTo(slotx, sloty, duration = 0.4-r.randint(0,3)/10)
    p.click(slotx,sloty)
    time.sleep(0.5)
    
# # Uncomment this section if this part of the form is not already filled -------------
# #click my name to autofill
# next = [512,563,579,598]
# move_n_click(next)
# time.sleep(0.3)

# #click dropdown to confirm for location
# next = [581,599,767,774]
# move_n_click(next)
# time.sleep(0.1)

# #click YES to confirm for location
# next = [496,596,819,833]
# move_n_click(next)
# time.sleep(0.1)

# #click dropdown to confirm no covid symptoms
# next = [575,599,959,968]
# move_n_click(next)
# time.sleep(0.1)

# #click NO to confirm no covid symptoms
# next = [495,579,1018,1031]
# move_n_click(next)
# time.sleep(0.1)
# # ----------------------------------------------------------------------------------

time.sleep(0.3)
p.scroll(-r.randint(600,1000))
#time.sleep(0.2)

#click continue
next = [587,706,944,968]
move_n_click(next)
time.sleep(0.5)

p.scroll(-r.randint(600,800))
time.sleep(0.2)
p.scroll(-r.randint(600,800))
time.sleep(0.2)
# p.scroll(-r.randint(600,800))
# time.sleep(0.2)

#click agree checkbox
next = [496,501,669,674]
move_n_click(next)
time.sleep(0.1)

#click captcha
next = [486,502,778,792]
move_n_click(next)
time.sleep(0.8)

#click complete booking
nextx = r.randint(595,718)
nexty = r.randint(860,873)
p.moveTo(nextx, nexty, duration = 0.4-r.randint(0,3)/10)
# p.click(nextx,nexty)
# time.sleep(0.3)
# p.click(nextx,nexty)
# time.sleep(0.3)
# p.click(nextx,nexty)
# time.sleep(0.3)

# change the "from_" number to your Twilio number and the "to" number to the phone number you signed up for Twilio with, or upgrade your account to send SMS to any phone number
#client.messages.create(to = "+17789852567", from_ = "+12564004669", body = "Your Code Did A Thing and Needs Checking!")

yeppers
